In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

loader = TextLoader("files/document.txt")
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
)
documents = loader.load_and_split(text_splitter=text_splitter)

underlying_embeddings = OpenAIEmbeddings()
store = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(underlying_embeddings, store)

db = FAISS.from_documents(documents, cached_embeddings)
retriever = db.as_retriever()

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1000, return_messages=True)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def save_memory(question, response):
    memory.save_context(
        {"input": question},
        {"output": response},
    )

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        You are a helpful AI that provides answer using only the following document. If you don't know the answer just say 'I don't know'. Don't forget to say 'I don't know' if you don't know the answer. Here is the document:
        ---
        {document}
        """
    ),
    ("system", "Now conversation started."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = {"document": retriever, "question": RunnablePassthrough()} | RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    save_memory(question, result.content)
    print(result.content)

In [2]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia")
invoke_chain("What was my first question")

According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
The message he wrote on the table was: "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE".
Julia is a character mentioned in the document who is associated with the protagonist and plays a significant role in the story.
Your first question was about Aaronson's guilt.
